# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006, and
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro)
1. [Step 1](#rderiv): ${\rm d}r / {\rm d}t$
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant.

We start with the following parameters:
1. a Cartesian tortoise separation vector $\left( x, y, z \right)$ (in units of total mass),
1. a Cartesian momentum vector ${\bf p^{*}}$ (in units of reduced mass),
1. a spin vector ${\bf S}_{1}$ (in units of total mass squared),
1. a spin vector ${\bf S}_{2}$ (in units of total mass squared),
1. orbital phase (in radians), and
1. $\alpha \cdot \cos\left( \iota \right)$ where $\iota$ is the inclination of the system.

<a id='rderiv'></a>

# ${\rm d}r / {\rm d}t$ \[Back to [top](#toc)\]
$$\label{rderiv}$$

Line 15 of LALSpinPrecHcapRvecDerivative_v3opt.c claims that we're first going to compute [BB2010](https://arxiv.org/abs/0912.3517) Equation (A4), but I don't believe that's the case.  

We need the following quantities:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \mu &= \frac{ m_{1} m_{2} }{ M } \\
    \eta &= \frac{ \mu }{ M } \\
    \xi &= 1 \\
    r^{2} &= x^{2} + y^{2} + z^{2} \\
    r &= \sqrt{r^{2}} \\
    u &= \frac{ 1 }{ r }
\end{align*}

In [1]:
%%writefile SEOBNR/initial_conditions.txt
M = m1 + m2
mu = m1*m2/M
eta = mu/M
xi = 1
rsq = x*x + y*y + z*z
r = sqrt(rsq)
u = 1/r

Overwriting SEOBNR/initial_conditions.txt


<a id='skerr'></a>

# ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }.
\end{equation*}

In [2]:
%%writefile -a SEOBNR/initial_conditions.txt

SKerr1 = S1x + S2x
SKerr2 = S1y + S2y
SKerr1 = S1z + S2z
asq = (SKerr1*SKerr1 + SKerr2*SKerr2 + SKerr3*SKerr3)/(M*M)

Appending to SEOBNR/initial_conditions.txt


<a id='w'></a>

# $\varpi$ \[Back to [top](#toc)\]
$$\label{w}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.7)

\begin{equation*}
    \varpi^{2} = r^{2} + a^{2}.
\end{equation*}

In [3]:
%%writefile -a SEOBNR/initial_conditions.txt

w2 = rsq + asq

Appending to SEOBNR/initial_conditions.txt


<a id='Dinv'></a>

# $D^{-1}$ \[Back to [top](#toc)\]
$$\label{Dinv}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.82)

\begin{equation*}
    D^{-1} = 1 + \log \left[ 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3} \right].
\end{equation*}

In [4]:
%%writefile -a SEOBNR/initial_conditions.txt

Dinv = 1 + sp.log(1 + 6*eta*u*u + 2(26 - 3*eta)*eta*u*u*u)

Appending to SEOBNR/initial_conditions.txt


<a id='deltaubar'></a>

# $\bar{\Delta}_{u}$ \[Back to [top](#toc)\]
$$\label{deltaubar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.75)

\begin{equation*}
    \bar{\Delta}_{u} = \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K -1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }.
\end{equation*}

In [5]:
%%writefile -a SEOBNR/initial_conditions.txt

Deltaubar = asq*u*u/(M*M) + 2*u/(eta*K - 1) + 1/((eta*K - 1)*(eta*K - 1))

Appending to SEOBNR/initial_conditions.txt


<a id='deltau'></a>

# $\Delta_{u}$ \[Back to [top](#toc)\]
$$\label{deltau}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.73)

\begin{equation*}
    \Delta_{u} = \bar{\Delta}_{u} \left[ 1 + \eta \Delta_{0} + \eta \log \left( 1 + \Delta_{1} u + \Delta_{2} u^{2} + \Delta_{3} u^{3} + \Delta_{4} u^{4} \right) \right].
\end{equation*}

<font color='red'>Note that in LALSpinPrechcapRvecDerivative_v3opt.c line 77 we take an absolute value.</font>

In [6]:
%%writefile -a SEOBNR/initial_conditions.txt

Deltau = Deltaubar*(1 + eta*Delta0 + eta*sp.log(1 + Delta1*u + Delta2*u*u
                                                + Delta3*u*u*u + Delta4*u*u*u*u))

Appending to SEOBNR/initial_conditions.txt


<a id='deltat'></a>

# $\Delta_{t}$ \[Back to [top](#toc)\]
$$\label{deltat}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.71)

\begin{equation*}
    \Delta_{t} = r^{2} \Delta_{u}.
\end{equation*}

In [7]:
%%writefile -a SEOBNR/initial_conditions.txt

Deltat = r*r*Deltau

Appending to SEOBNR/initial_conditions.txt


<a id='deltar'></a>

# $\Delta_{r}$ \[Back to [top](#toc)\]
$$\label{deltar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.38)

\begin{equation*}
    \Delta_{r} = \Delta_{t} D^{-1}.
\end{equation*}

In [8]:
%%writefile -a SEOBNR/initial_conditions.txt

Deltar = Deltat*Dinv

Appending to SEOBNR/initial_conditions.txt


<a id='xi'></a>

# $\xi$ \[Back to [top](#toc)\]
$$\label{xi}$$

Inverting [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (28),

\begin{equation*}
    \xi = \frac{ \sqrt{ \Delta_{t} \Delta_{r} } }{ \varpi^{2} }.
\end{equation*}

In [9]:
%%writefile -a SEOBNR/initial_conditions.txt

xi = sp.sqrt(Deltat*Deltar)/w2

Appending to SEOBNR/initial_conditions.txt


<a id='k'></a>

# $K$ \[Back to [top](#toc)\]
$$\label{k}$$

Following the discussion in the two paragrahs after [BB2010](https://arxiv.org/abs/0912.3517) Equation (6.11), we have

\begin{equation*}
    K = 1.4467 \left( 1 - 4 \eta \right)^{2} + 4 \left( 1 - 2 \eta \right) \eta.
\end{equation*}

In [10]:
%%writefile -a SEOBNR/initial_conditions.txt

K = 1.4467*(1 - 4*eta)*(1 - 4*eta) + 4*(1 - 2*eta)*eta

Appending to SEOBNR/initial_conditions.txt


<a id='calibcoeffs'></a>

# Calibration Coefficients $\Delta_{i}$, $i \in \left\{0, 1, 2, 3, 4\right\}$ \[Back to [top](#toc)\]
$$\label{calibcoeffs}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.77)--(5.81), we have

\begin{align*}
    \Delta_{0} &= K \left( \eta K - 2 \right) \\
    \Delta_{1} &= -2 \left( \eta K - 1 \right) \left( K + \Delta_{0} \right) \\
    \Delta_{2} &= \frac{1}{2} \Delta_{1} \left( -4 \eta K + \Delta_{1} + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{0}\\
    \Delta_{3} &= \frac{1}{3} \left[ -\Delta_{1}^{3} + 3 \left( \eta K - 1 \right) \Delta_{1}^{2} + 3 \Delta_{2} \Delta_{1} - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_{2} + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_{1} \right] \\
    \Delta_{4} &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_{1}^{2} - 2 \Delta_{2} \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_{1}^{4} - 8 \left( \eta K - 1 \right) \Delta_{1}^{3} - 12 \Delta_{2} \Delta_{1}^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_{2} + \Delta_{3} \right] \Delta_{1} \right.\\
        &\left.\ \ \ \ \ + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_{2}^{2} - 4 \Delta_{3} \left( \eta K - 1 \right) \right] \right\}
\end{align*}

In [11]:
%%writefile -a SEOBNR/initial_conditions.txt

Delta0 = K*(eta*K - 2)
Delta1 = -2*etaKminus1*(K + Delta0)
Delta2 = sp.Rational(1,2)*Delta1*(-4*eta*K + Delta1 + 4) - a2M2inv*etaKminus1*etaKminus1*Delta0
Delta3 = sp.Rational(1,3)*(-Delta1***3 + 3*etaKminus1*Delta1**2 + 3*Delta2*Delta1 - 6*etaKminus1*(-eta*K + Delta2 + 1)
                           - 3*a2M2inv*etaKminus1**2*Delta1))
Delta4 = sp.Rational(1,12)*(6*a2M2inv*(Delta1**2 - 2*Delta2)*etaKminus1**2 + 3*Delta1**4 - 8*etaKminus1*Delta1**3
                           -12*Delta2*Delta1**2 + 12*(2*etaKminus1*Delta2 + Delta3)*Delta1
                           + 12*(sp.Rational(94,3) - sp.Rational(41,32)*pi**2)*etaKminus1**2
                           + 6*(Delta2**2 - 4*Delta3*etaKminus1))

Appending to SEOBNR/initial_conditions.txt


<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [12]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Initial_Conditions.ipynb to latex
[NbConvertApp] Writing 37308 bytes to Tutorial-SEOBNR_Initial_Conditions.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
